<a href="https://colab.research.google.com/github/donghyundavidchoi/Data_Analysis/blob/main/%EB%AA%A8%EB%8D%B8%EB%A7%81/%EC%A0%95%EC%8B%9C%EB%B0%B0%EC%86%A1%EC%98%88%EC%B8%A1%EB%AA%A8%EB%8D%B8_RandomForestClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 정시배송예측모델

In [1]:
!gdown --fuzzy https://drive.google.com/file/d/1txRVXp-Eh222PzIeTQ-fuAoUYFR9f7q-/view?usp=sharing
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Downloading...
From: https://drive.google.com/uc?id=1txRVXp-Eh222PzIeTQ-fuAoUYFR9f7q-
To: /content/kaggle.json
100% 73.0/73.0 [00:00<00:00, 90.3kB/s]


In [2]:
!kaggle datasets download -d prachi13/customer-analytics

  0% 0.00/121k [00:00<?, ?B/s]
100% 121k/121k [00:00<00:00, 62.7MB/s]


In [3]:
!unzip /content/customer-analytics.zip

Archive:  /content/customer-analytics.zip
  inflating: Train.csv               


In [333]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2022)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test

In [334]:
df = pd.read_csv("/content/Train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Reached.on.Time_Y.N', id_name='ID')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8799, 11), (2200, 11), (8799, 2), (2200, 2))

-----------

# EDA

In [335]:
X_train

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
8439,8440,B,Ship,4,1,284,4,low,F,4,4795
10848,10849,D,Ship,3,2,201,4,medium,M,10,5565
9477,9478,B,Ship,4,4,181,3,medium,F,9,4425
2967,2968,B,Ship,6,2,236,4,low,F,61,3140
6963,6964,B,Road,5,1,282,4,low,F,9,4766
...,...,...,...,...,...,...,...,...,...,...,...
6384,6385,D,Ship,3,1,166,2,high,F,6,5905
4720,4721,C,Ship,4,2,152,2,low,M,7,5026
173,174,F,Ship,3,2,270,3,low,F,63,2751
1244,1245,A,Flight,4,2,136,2,medium,F,17,1285


In [336]:
X_train.isna().sum()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64

In [337]:
X_train.nunique()

ID                     8799
Warehouse_block           5
Mode_of_Shipment          3
Customer_care_calls       6
Customer_rating           5
Cost_of_the_Product     215
Prior_purchases           8
Product_importance        3
Gender                    2
Discount_offered         65
Weight_in_gms          3761
dtype: int64

-------

# 라벨인코딩 진행
# ID 삭제
# Warehouse_block : 창고 내 섹터는 큰 영향을 미치지 않을 것이라 판단하여 삭제
# Prior_Purchases : 각각의 배송은 독립사건임. 만약 영향을 미쳤더라도 이미 고객 등급에 선반영되었을 것
# Discount_offered : 개별 상품에 대한 이슈.
# Gender : 성별에 따라 주문한 상품의 종류가 다를 수 있지만 상품의 무게가 있는 만큼 크게 영향을 미칠 요소는 아니라고 판단 

In [338]:
from sklearn.preprocessing import LabelEncoder

In [339]:
LE = LabelEncoder()
columns = ['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender']
for col in columns:
  X_train[col] = LE.fit_transform(X_train[col])
  X_test[col] = LE.transform(X_test[col])

In [340]:
X_train_ID = X_train['ID']
X_train = X_train.drop(columns='ID', axis=1)
X_test_ID = X_test['ID']
X_test = X_test.drop(columns='ID', axis=1)

In [341]:
X_train = X_train.drop(columns='Warehouse_block', axis=1)
X_test = X_test.drop(columns='Warehouse_block', axis=1)
X_train = X_train.drop(columns='Prior_purchases', axis=1)
X_test = X_test.drop(columns='Prior_purchases', axis=1)
X_train = X_train.drop(columns='Discount_offered', axis=1)
X_test = X_test.drop(columns='Discount_offered', axis=1)
X_train = X_train.drop(columns='Gender', axis=1)
X_test = X_test.drop(columns='Gender', axis=1)

In [342]:
X_train.describe()

,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Product_importance,Weight_in_gms
count,8799.000000,8799.000000,8799.000000,8799.000000,8799.000000,8799.000000
mean,1.520059,4.051369,2.991817,209.800318,1.349131,3633.145130
std,0.752975,1.141739,1.411938,48.186779,0.630860,1637.717073
min,0.000000,2.000000,1.000000,96.000000,0.000000,1001.000000
25%,1.000000,3.000000,2.000000,169.000000,1.000000,1837.000000
50%,2.000000,4.000000,3.000000,213.000000,1.000000,4152.000000
75%,2.000000,5.000000,4.000000,250.000000,2.000000,5055.000000
max,2.000000,7.000000,5.000000,310.000000,2.000000,7846.000000


In [343]:
X_train.corr()

,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Product_importance,Weight_in_gms
Mode_of_Shipment,1.000000,-0.016535,-0.002091,0.015158,0.000330,0.007412
Customer_care_calls,-0.016535,1.000000,0.015984,0.327246,0.008078,-0.272845
Customer_rating,-0.002091,0.015984,1.000000,0.008005,0.002314,-0.006745
Cost_of_the_Product,0.015158,0.327246,0.008005,1.000000,0.006104,-0.131526
Product_importance,0.000330,0.008078,0.002314,0.006104,1.000000,-0.001761
Weight_in_gms,0.007412,-0.272845,-0.006745,-0.131526,-0.001761,1.000000


In [344]:
y_train

,ID,Reached.on.Time_Y.N
8439,8440,1
10848,10849,0
9477,9478,1
2967,2968,1
6963,6964,0
...,...,...
6384,6385,1
4720,4721,0
173,174,1
1244,1245,1


----------

# 라벨 인코더가 사용되고 이진부류임으로 랜덤포레스트분류 모델 사용
# 과적합 확인을 위해 train 데이터를 train 데이터와 validation 데이터로 한 번 더 분리하여 모델링 및 과적합 검증 시도

In [345]:
from sklearn.ensemble import RandomForestClassifier

In [346]:
from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(X_train,y_train,test_size=0.2,random_state=2022)

In [436]:
model = RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_split=4, min_samples_leaf=20, random_state=2022)
model.fit(X_tr, y_tr['Reached.on.Time_Y.N'])
print(model.score(X_tr, y_tr['Reached.on.Time_Y.N']))
print(model.score(X_val, y_val['Reached.on.Time_Y.N']))

0.6931382298621963
0.69375


--------

# 학습된 모델로 예측된 결과 csv 파일로 저장

In [437]:
prediction = model.predict_proba(X_test)

In [438]:
output = pd.DataFrame({'ID':X_test_ID, 'Reached.on.Time_Y.N':prediction[:,1]})

In [439]:
output

,ID,Reached.on.Time_Y.N
8284,8285,0.462073
10191,10192,0.467888
8674,8675,0.435421
5752,5753,0.455196
447,448,0.931414
...,...,...
7192,7193,0.617758
4444,4445,0.453034
7389,7390,0.466763
9976,9977,0.453602


In [440]:
output.to_csv('20220529.csv',index=False)

In [441]:
data = pd.read_csv('/content/20220529.csv')
data

,ID,Reached.on.Time_Y.N
0,8285,0.462073
1,10192,0.467888
2,8675,0.435421
3,5753,0.455196
4,448,0.931414
...,...,...
2195,7193,0.617758
2196,4445,0.453034
2197,7390,0.466763
2198,9977,0.453602


--------

# 최종 점수

In [442]:
round(model.score(X_test, y_test['Reached.on.Time_Y.N']) * 100, 2)

67.05